In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

df = pd.read_parquet('./data')
print(f'total dataset: {len(df)} rows')

## filters
df = df[df['stabilized_spread'] < 10]  # nem tul nagy spread
df = df[df['stabilized_spread'] > 0.5]  # nem tul kis spread
# df = df[abs(df['last_price_delta_since_stabilized']) > 0.1]
# df = df[abs(df['last_price_delta_since_stabilized']) < 8]

# df = df[df['stabilized_amount_mean'] < 0.4]  # nem tul nagy tradek
# df = df[df['stabilized_nr_trades'] < 300]  # nem tul sok trade

df.sort_index(axis=1, inplace=True)

for col in df.columns:
    # if 'nr_trades' in col or 'price_delta' in col and not col.startswith('last'):
    #     df.drop(col, axis=1, inplace=True)
    if 'past' in col:
        df.drop(col, axis=1, inplace=True)
    if '_spread' == col[1:]:
        df.drop(col, axis=1, inplace=True)


y = df['last_price_delta_since_stabilized']
X = df.drop('last_price_delta_since_stabilized', axis=1)

X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.75, test_size=0.25, random_state=0)
X_train

total dataset: 121438 rows


,0_amount_mean,0_nr_trades,0_price_delta,1_amount_mean,1_nr_trades,1_price_delta,2_amount_mean,2_nr_trades,2_price_delta,3_amount_mean,...,3_price_delta,4_amount_mean,4_nr_trades,4_price_delta,stabilized_amount_mean,stabilized_at_ms,stabilized_gasp,stabilized_nr_trades,stabilized_spread,wave_direction
index,,,,,,,,,,,,,,,,,,,,,
0,0.0389,15.0,0.64,0.0483,16.0,0.64,0.0455,17.0,0.64,0.0432,...,0.64,0.0405,20.0,0.64,0.0443,485,-1.4448,30.0,1.37,max
0,0.0140,5.0,1.01,0.0118,6.0,0.99,0.0102,7.0,0.96,0.0097,...,0.59,0.0104,26.0,0.19,0.0103,177,0.0446,28.0,2.09,min
0,0.0037,24.0,0.09,0.0036,25.0,0.09,0.0038,26.0,0.09,0.0037,...,0.08,0.0035,29.0,0.02,0.0036,723,-0.5509,31.0,1.31,max
0,0.0055,4.0,0.79,0.0153,13.0,0.03,0.0148,15.0,0.03,0.0140,...,0.02,0.0186,19.0,0.01,0.0179,337,-0.8569,21.0,1.47,max
0,0.0032,3.0,0.61,0.0089,10.0,0.23,0.0076,12.0,0.10,0.0074,...,0.01,0.0073,16.0,0.00,0.0068,257,0.5210,19.0,0.65,min
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,0.0026,19.0,0.09,0.0034,21.0,0.09,0.0033,22.0,0.09,0.0032,...,0.09,0.0031,24.0,0.02,0.0032,595,-0.1519,25.0,1.15,max
0,0.0138,9.0,0.19,0.0163,11.0,0.03,0.0145,13.0,0.01,0.0132,...,0.00,0.0164,17.0,0.00,0.0157,1098,-0.4763,18.0,0.52,None
0,0.0071,17.0,0.16,0.0068,18.0,0.16,0.0066,20.0,0.16,0.0065,...,0.16,0.0062,22.0,0.16,0.0058,607,-0.2220,24.0,0.76,max


In [2]:
from spreadsurfer.price_engine import FeatureEngineer
from catboost import CatBoostRegressor
from sklearn.preprocessing import StandardScaler
# from sklearn.ensemle import RandomForestRegressor

model = CatBoostRegressor(learning_rate=0.01, depth=7, loss_function='MAPE', random_state=0, verbose=False, iterations=1500)
model2 = CatBoostRegressor(learning_rate=0.01, depth=7, loss_function='RMSE', random_state=0, verbose=False, iterations=1500)
trained_model = model

pipeline = Pipeline(steps=[
    ('preprocessor', FeatureEngineer()),
    ('model', model)
])

pipeline2 = Pipeline(steps=[
    ('preprocessor', FeatureEngineer()),
    ('model', model2)
])

pipeline.fit(X_train, y_train)
pipeline2.fit(X_train, y_train)
score = pipeline.score(X_valid, y_valid)
score2 = pipeline2.score(X_valid, y_valid)
display(score)
display(score2)


2023-02-03 17:27:09.603 | INFO     | spreadsurfer.balance:<module>:9 - balance config: {'panic_below_total': 390, 'panic_countdown_from': 8}
2023-02-03 17:27:09.604 | INFO     | spreadsurfer.waves:<module>:15 - wave detection config: {'min_length': 6, 'investigate_length': 5, 'stabilized_threshold': 0.01, 'max_delta_ms_to_create_order': 500, 'collect_last_n_frames': 6, 'collect_last_n_wave_prices': 10}
2023-02-03 17:27:09.693 | INFO     | spreadsurfer.orders:<module>:18 - order config: {'orders_disabled': False, 'test_mode': True, 'max_nr_orders_limited': True, 'max_nr_orders_created': 20, 'base_amount': 0.0015, 'skip_order_on_spread_below': 0.2, 'skip_order_on_spread_above': 10, 'low_spread_limit': 0.2, 'recv_window': 100, 'cancel_far_order_after_ms': 400}


0.1626978784278862

0.3128700920573255

In [517]:
import catboost
w = catboost.MetricVisualizer('./models', subdirs=True)
w.start()

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

boo


In [37]:
from sklearn.model_selection import GridSearchCV

do_grid_search = False

if do_grid_search:
    X = FeatureEngineer().transform(X)
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)

    parameters = {'depth': [6, 7, 8],
                  'learning_rate': [0.015, 0.025],
                  'iterations': [500, 1000, 1500]
                  }
    grid = GridSearchCV(estimator=model, param_grid=parameters, cv=2, n_jobs=-1)
    grid.fit(X_train, y_train)

    print(" Results from Grid Search ")
    print("\n The best estimator across ALL searched params:\n", grid.best_estimator_)
    print("\n The best score across ALL searched params:\n", grid.best_score_)
    print("\n The best parameters across ALL searched params:\n", grid.best_params_)

 Results from Grid Search 

 The best estimator across ALL searched params:

 The best score across ALL searched params:
 0.41200864218441446

 The best parameters across ALL searched params:
 {'depth': 7, 'iterations': 1500, 'learning_rate': 0.025}


In [139]:
from spreadsurfer import now_isoformat

save = True
if save:
    model.save_model(f'./models/{now_isoformat()}-mape-score-{1000 * round(score, 3)}.cat')
    model2.save_model(f'./models/{now_isoformat()}-rmse-score-{1000 * round(score2, 3)}.cat')


In [1]:
with open("console.log","r") as f:
    input_from_log = ''.join([x for x in f.readlines() if 'collected' in x][-100:])

def predict_samples():
    mae = 0
    mae2 = 0
    count = 0
    for line in input_from_log.split('\n'):
        if not line: continue
        count += 1
        json = '{' + line.split('{')[1]
        json = json.replace('nan', '0')
        sample = pd.DataFrame([eval(json)])

        real = sample.last_price_delta_since_stabilized[0]
        if real < 0.1: continue
        print('expected: ', real)
        sample.drop('last_price_delta_since_stabilized', axis=1, inplace=True)

        guess = pipeline.predict(sample)[0]
        guess2 = pipeline2.predict(sample)[0]
        print('predict: ', guess)
        print('predict2: ', guess2)
        mae += abs(real - guess)
        mae2 += abs(real - guess2)
        print()
    display(f'MAE: {mae / count}')
    display(f'MAE2: {mae2 / count}')
predict_samples()

NameError: name 'pd' is not defined

In [100]:
sample = pd.DataFrame([
    {'0_amount_mean': 0.007, '0_nr_trades': 12.0, '0_price_delta': 0.91, '0_spread': 1.15, '1_amount_mean': 0.007, '1_nr_trades': 13.0, '1_price_delta': 0.91, '1_spread': 1.52, '2_amount_mean': 0.006, '2_nr_trades': 14.0,
     '2_price_delta': 0.91, '2_spread': 1.52, '3_amount_mean': 0.006, '3_nr_trades': 16.0, '3_price_delta': 0.69, '3_spread': 1.74, '4_amount_mean': 0.005, '4_nr_trades': 17.0, '4_price_delta': 0.69, '4_spread': 1.74,
     'last_price_delta_since_stabilized': -0.87, 'past_final_price_0': -0.11999999999898137, 'past_final_price_1': 0.25, 'past_final_price_2': -0.020000000000436557, 'past_final_price_3': -1.7400000000016007,
     'past_final_price_4': 0.06999999999970896, 'past_final_price_5': -2.7299999999995634, 'past_final_price_6': -3.9099999999998545, 'past_final_price_7': 3.360000000000582, 'past_final_price_8': -0.06000000000130967,
     'past_final_price_9': 0.0, 'stabilized_amount_mean': 0.007, 'stabilized_at_ms': 369, 'stabilized_nr_trades': 18.0, 'stabilized_spread': 2.43, 'wave_direction': 'max'}
])

print('expected: ', sample.last_price_delta_since_stabilized.real())
sample.drop('last_price_delta_since_stabilized', axis=1, inplace=True)

from timeit import timeit

timeit(lambda: pipeline.predict(sample), number=1) * 1000
print('predict: ', pipeline.predict(sample)[0])


AttributeError: 'Series' object has no attribute 'real'

In [154]:
from glob import glob

cat_filename = glob('*.cat')[0]
cat_filename

model = CatBoostRegressor()
model.load_model(fname=cat_filename)
print(f'catboost model loaded from {cat_filename}')


pipeline = Pipeline(steps=[
    ('preprocessor', FeatureEngineer()),
    ('model', model)
])

predict_samples()


catboost model loaded from 2023-02-01T09:14:50-rmse-score-291.0.cat
expected:  0.15
predict:  0.598326925615093
predict2:  0.5982384053273737

expected:  0.63
predict:  0.5701546470393212
predict2:  0.5327435323645455

expected:  0.92
predict:  0.2571625478008876
predict2:  0.26213133691454216



'MAE: 0.07318810817343027'

'MAE2: 0.07521022100301789'